In [4]:
import pandas as pd
import numpy as np
import re 

from collections import Counter
import spacy #lemmatization 
#pre spacy treba v terminali doinstalovat 2 prikazy:
#pip3 install spacy
#python3 -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm") #nlp = spacy.load("en_core_web_lg") finalne

from sklearn.feature_extraction.text import CountVectorizer
from sklearn. feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import text 

In [5]:
data = pd.read_csv("sample_2007_2022.csv", sep = '\t' ,on_bad_lines='skip', low_memory=False)

In [6]:
data

,pmid,year,month,title
0,17690230,2007,Aug,"The visual pigments of a deep-sea teleost, the..."
1,17682436,2007,Jul,[Primary liver carcinoid tumor complicated wit...
2,17875888,2007,Sep,Diffuse nodules in a woman with renal failure:...
3,17628980,2007,Apr,[Feature of ACh-sensitive potassium current in...
4,17666563,2007,Aug,Helicobacter pylori eradication in the managem...
...,...,...,...,...
1638182,36452147,2022,NaN,Meta-analysis of substitution value of maize w...
1638183,35583455,2022,Aug,Intraplaque calcium and its relation with the ...
1638184,35565964,2022,Apr,Mechanistic Insights into the Ameliorating Eff...
1638185,35080294,2022,01,Controversies in the treatment of mild asthma....


In [16]:
data["title"] = data["title"].astype(str)  #zmenime stlpcec na typ str 
data['year'] = data['year'].astype('Int64')   #zmenime typ stlpca year na integer

In [17]:
#preprocessing text
def clean_text(text):
  text = re.sub(r'[^A-Za-z0-9 ]+',' ', text) #odstranime vsetko okrem pismen, cislic a medzier 
  return text
data["title_cleaned"] = data["title"].str.lower() #zmenime text na male pismena 
data['title_cleaned'] = data['title_cleaned'].apply(clean_text) #vycistime 

In [18]:
df = data.copy(deep = True)

In [61]:
#tuto musime data rozdelit na 3 mensie dataframe po 550 000 zaznamov

data = df[:550000]
#data = df[550000:1100000]
#data = df[1100000:]

In [62]:
data["lemma"] = "" #new column - column title after lemmatization 

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_38365/2638881957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["lemma"] = "" #new column - column title after lemmatization


In [63]:
#pridame stlpec lemmatization po jednotlivych vetach  

for index_row, row in data.iterrows():
    sentence_lemma = []
    #print(index_row, " ",row['title_cleaned'])
    data_tokens_spacy = nlp(row['title_cleaned'])
    for token in data_tokens_spacy:   
      sentence_lemma.append(token.lemma_)
    #print(sentence_lemma)
    data.at[index_row,"lemma"] = " ".join(sentence_lemma) 
    

In [65]:
#ulozime data do suboru oddelene tabulatorom

data.to_csv("2007/lemma_2007_22_p1.csv", sep = '\t', index = False)
#data.to_csv("2007/lemma_2007_22_p2.csv", sep = '\t', index = False)
#data.to_csv("2007/lemma_2007_22_p3.csv", sep = '\t', index = False)

In [66]:
#skombinujeme english a nase stopwords
stopwords_table = pd.read_csv('stop_words.txt')
custom_stopwords = list(stopwords_table['based'].values)
stop_words = text.ENGLISH_STOP_WORDS.union(custom_stopwords)

In [67]:
#pridame stlpec z lemma okrem stopwords
def rem_stop(text):
    return  ' '.join([word for word in text.split() if not word in stop_words])

#data['lemma'] = data['lemma'].astype(str)
data['lemma_without_stopwords'] = data['lemma'].apply(rem_stop)

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_38365/1768465911.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lemma_without_stopwords'] = data['lemma'].apply(rem_stop)


In [68]:
#vytvorime count vectorizer a urobime vektor dat z lemma
cv = CountVectorizer (max_df=0.90,stop_words = stop_words, max_features=10000)
word_count_vector = cv.fit_transform(data['lemma'].values.astype('U'))

In [69]:
#vytvorime tfid model a nafitujeme data z countvectorizer
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [70]:
#funkcie na triedenie slov podla ich tfidf hodnot
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_keywords(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    feature_vals = []
    for idx, score in sorted_items:
        feature_vals.append(feature_names[idx])
    return feature_vals

In [71]:
#ziskame top 5 klucovych slov zo stlpca lemma s najvacsou tfidf hodnotou
idf_keywords = []
data['lemma'] = data['lemma'].values.astype('U')
feature_names = cv.get_feature_names_out()

for i in range(data['lemma'].size):
    tf_idf_vector = tfidf_transformer.transform(cv.transform([data.iloc[i]['lemma']]))
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_keywords(feature_names, sorted_items, 5)
    #print("ahoj")
    idf_keywords.append(" ".join(keywords))

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_38365/846251361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lemma'] = data['lemma'].values.astype('U')


In [73]:
#pridame stlpec s vybranymi keywords
data['tfidf_keywords'] = idf_keywords
data[:15]

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_38365/2242405160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tfidf_keywords'] = idf_keywords


,pmid,year,month,title,title_cleaned,lemma,lemma_without_stopwords,tfidf_keywords
1100000,31788588,2019,Nov,Combining Charge Density Analysis with Machine...,combining charge density analysis with machine...,combine charge density analysis with machine l...,combine charge density machine learning tool i...,charge investigate density machine tool
1100001,31793200,2019,Dec,Comparative Proteomics Analysis Reveals Roles ...,comparative proteomics analysis reveals roles ...,comparative proteomic analysis reveal role for...,comparative proteomic reveal fadd regulation e...,proteolysis embryonic fibroblast proteomic met...
1100002,31321074,2019,NaN,The new BMJ Open Sport & Exercise Medicine in ...,the new bmj open sport exercise medicine in ...,the new bmj open sport exercise medicine in...,bmj open sport exercise medicine everchanging ...,publish sport landscape open exercise
1100003,30898423,2019,Oct,Direct Intraoperative Neurologic Assessment to...,direct intraoperative neurologic assessment to...,direct intraoperative neurologic assessment to...,direct intraoperative neurologic monitor spina...,thoracoabdominal neurologic intraoperative end...
1100004,30728047,2019,Feb,Focal adhesion kinase (FAK) activation by estr...,focal adhesion kinase fak activation by estr...,focal adhesion kinase fak activation by es...,focal adhesion kinase fak estrogen involve gpe...,fak estrogen focal adhesion triple
1100005,30393178,2019,Jan,Transcriptome analysis of spleen reveals the s...,transcriptome analysis of spleen reveals the s...,transcriptome analysis of spleen reveal the si...,transcriptome spleen reveal signal transductio...,aeromonas transduction spleen toll transcriptome
1100006,30816387,2019,Apr,Endohedral metal-nitride cluster ordering in m...,endohedral metal nitride cluster ordering in m...,endohedral metal nitride cluster order in meta...,endohedral metal nitride cluster order metallo...,nitride ni theoretical order crystal
1100007,31012832,2019,04,Overlapping surgeries: defining the 'critical ...,overlapping surgeries defining the critical ...,overlap surgery define the critical portio...,overlap surgery define critical portion procedure,portion overlap define procedure critical
1100008,30509729,2019,Jan,Linking microbial respiratory activity with ph...,linking microbial respiratory activity with ph...,link microbial respiratory activity with phosp...,link microbial respiratory phospholipid fatty ...,bioreactor phospholipid biofilm microbial fatty
1100009,31869370,2019,NaN,Lead-I ECG for detecting atrial fibrillation i...,lead i ecg for detecting atrial fibrillation i...,lead I ecg for detect atrial fibrillation in p...,lead I ecg detect atrial fibrillation attend c...,ecg attend pulse economic fibrillation


In [74]:
#overime title a jeho key words
print(data.iloc[0]['title'])
print(data.iloc[0]['tfidf_keywords'])

Combining Charge Density Analysis with Machine Learning Tools To Investigate the Cruzain Inhibition Mechanism.
charge investigate density machine tool


In [75]:
#ulozime vyslednu tabulku do textoveho suboru
data.to_csv("2007/2007_22_p1.csv", sep = '\t', index = False)
#data.to_csv("2007/2007_22_p2.csv", sep = '\t', index = False)
#data.to_csv("2007/2007_22_p3.csv", sep = '\t', index = False)

In [77]:
#nacitame a zlepime 3tabulky
df1 = pd.read_csv("2007/2007_22_p1.csv", sep = '\t' ,on_bad_lines='skip')
df2 = pd.read_csv("2007/2007_22_p2.csv", sep = '\t' ,on_bad_lines='skip')
df3 = pd.read_csv("2007/2007_22_p3.csv", sep = '\t' ,on_bad_lines='skip')

df123 = pd.concat([df1, df2, df3])
df123

,pmid,year,month,title,title_cleaned,lemma,lemma_without_stopwords,tfidf_keywords
0,17690230,2007,Aug,"The visual pigments of a deep-sea teleost, the...",the visual pigments of a deep sea teleost the...,the visual pigment of a deep sea teleost the...,visual pigment deep sea teleost pearl eye scop...,teleost pearl pigment sea deep
1,17682436,2007,Jul,[Primary liver carcinoid tumor complicated wit...,primary liver carcinoid tumor complicated wit...,primary liver carcinoid tumor complicate wit...,liver carcinoid tumor complicate colon cancer,carcinoid complicate colon liver tumor
2,17875888,2007,Sep,Diffuse nodules in a woman with renal failure:...,diffuse nodules in a woman with renal failure ...,diffuse nodule in a woman with renal failure ...,diffuse nodule woman renal failure chronic top...,gout nodule diffuse failure renal
3,17628980,2007,Apr,[Feature of ACh-sensitive potassium current in...,feature of ach sensitive potassium current in...,feature of ach sensitive potassium current i...,feature ach sensitive potassium current outer ...,cochlea outer guinea hair potassium
4,17666563,2007,Aug,Helicobacter pylori eradication in the managem...,helicobacter pylori eradication in the managem...,helicobacter pylori eradication in the managem...,helicobacter pylori eradication idiopathic pul...,eradication helicobacter pylori idiopathic fib...
...,...,...,...,...,...,...,...,...
538182,36452147,2022,NaN,Meta-analysis of substitution value of maize w...,meta analysis of substitution value of maize w...,meta analysis of substitution value of maize w...,substitution value maize cassava manihot escul...,cassava substitution broiler maize chicken
538183,35583455,2022,Aug,Intraplaque calcium and its relation with the ...,intraplaque calcium and its relation with the ...,intraplaque calcium and its relation with the ...,intraplaque calcium relation progression carot...,carotid relation calcium progression
538184,35565964,2022,Apr,Mechanistic Insights into the Ameliorating Eff...,mechanistic insights into the ameliorating eff...,mechanistic insight into the ameliorate effect...,mechanistic insight ameliorate melanogenesis p...,mechanistic ameliorate melanoma derivative ins...
538185,35080294,2022,01,Controversies in the treatment of mild asthma....,controversies in the treatment of mild asthma ...,controversy in the treatment of mild asthma ...,controversy mild asthma novelty practical impl...,novelty controversy practical mild asthma


In [78]:
#ulozime jednu velku zlepenu tabulku
df123.to_csv("2007/2007_22.csv", sep = '\t', index = False)